## Extracción de datos

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [14]:
# Lista de URLs a scrapear
urls = [
    "https://sofifa.com/players?type=all&lg%5B0%5D=13&tm%5B0%5D=1&r=220069&set=true",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=2",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1795",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1925",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1808",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=5",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1799",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=7",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1792",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=8",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=95",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=9",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=10",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=11",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=13",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1796",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=17",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=18",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=19",
    "https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=110"
]

In [15]:
# Configuración de Selenium para Chrome en modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
# Opcional: agregar un user-agent para imitar un navegador real
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/115.0.0.0 Safari/537.36")

In [16]:
# Inicializar el WebDriver (asegúrate de tener chromedriver instalado y en tu PATH)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1920, 1080)


In [17]:
all_data = []  # Lista para almacenar los datos extraídos (cada fila es un diccionario)

# Recorrer cada URL y extraer la tabla
for url in urls:
    print(f"Procesando {url}")
    try:
        driver.get(url)
        # Esperar hasta que la tabla esté presente (máximo 10 segundos)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )
        
        # Obtener el contenido de la página y parsearlo con BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table")
        
        if table:
            # Extraer cabeceras de la tabla (<th>)
            headers = [th.text.strip() for th in table.find_all("th")]
            # Iterar sobre las filas de la tabla (<tr>)
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if cells:
                    cell_values = [cell.text.strip() for cell in cells]
                    # Crear un diccionario si el número de celdas coincide con el de cabeceras
                    if len(headers) == len(cell_values):
                        row_dict = dict(zip(headers, cell_values))
                        all_data.append(row_dict)
        else:
            print("No se encontró tabla en", url)
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
    # Pausa para evitar sobrecargar el servidor
    time.sleep(2)

driver.quit()

Procesando https://sofifa.com/players?type=all&lg%5B0%5D=13&tm%5B0%5D=1&r=220069&set=true
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=2
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1795
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1925
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1808
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=5
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1799
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=7
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=1792
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=8
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=95
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=9
Procesando https://sofifa.com/players?type=all&lg%5B%5D=13&tm%5B0%5D=10
Procesando https://sofifa.com/players?typ

In [21]:
# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(all_data)
df

,,Name,Age,Overall rating,Potential,Team & Contract,Value,Wage,Total stats
0,,T. Partey CDM CM,28,84,84,Arsenal\n2020 ~ 2024,€37M,€115K,2161
1,,M. Ødegaard CAM CM,22,83,89,Arsenal\n2021 ~ 2026,€58.5M,€95K,2104
2,,A. Lacazette ST,30,82,82,Arsenal\n2017 ~ 2022,€30M,€110K,2058
3,,B. Saka RM LM,19,82,89,Arsenal\n2018 ~ 2024,€59.5M,€58K,2091
4,,B. Leno GK,29,81,81,Arsenal\n2018 ~ 2023,€17.5M,€74K,1344
...,...,...,...,...,...,...,...,...,...
650,,T. Perry CM CAM,19,59,72,Wolverhampton Wanderers\n2018 ~ 2023,€525K,€6K,1667
651,,H. Birtwistle RWB RW,17,55,72,Wolverhampton Wanderers\n2021 ~ 2023,€300K,€500,1518
652,,J. Hodge CM CDM,18,55,72,Wolverhampton Wanderers\n2021 ~ 2024,€325K,€3K,1561
653,,O. Hesketh CM CAM,18,54,70,Wolverhampton Wanderers\n2019 ~ 2023,€275K,€3K,1582


In [22]:
df.drop(columns=df.columns[0], axis=1, inplace=True)
df

,Name,Age,Overall rating,Potential,Team & Contract,Value,Wage,Total stats
0,T. Partey CDM CM,28,84,84,Arsenal\n2020 ~ 2024,€37M,€115K,2161
1,M. Ødegaard CAM CM,22,83,89,Arsenal\n2021 ~ 2026,€58.5M,€95K,2104
2,A. Lacazette ST,30,82,82,Arsenal\n2017 ~ 2022,€30M,€110K,2058
3,B. Saka RM LM,19,82,89,Arsenal\n2018 ~ 2024,€59.5M,€58K,2091
4,B. Leno GK,29,81,81,Arsenal\n2018 ~ 2023,€17.5M,€74K,1344
...,...,...,...,...,...,...,...,...
650,T. Perry CM CAM,19,59,72,Wolverhampton Wanderers\n2018 ~ 2023,€525K,€6K,1667
651,H. Birtwistle RWB RW,17,55,72,Wolverhampton Wanderers\n2021 ~ 2023,€300K,€500,1518
652,J. Hodge CM CDM,18,55,72,Wolverhampton Wanderers\n2021 ~ 2024,€325K,€3K,1561
653,O. Hesketh CM CAM,18,54,70,Wolverhampton Wanderers\n2019 ~ 2023,€275K,€3K,1582


In [23]:
df.to_csv('premier_league_2022_wages_sofifa.csv', index=False)